# Comparing exapmle

In [25]:
import pandas as pd

import similarity.functions as f
import time
from comparing import ComparatorForDatasets
from comparing import CategoricalSimilarity
from similarity.DataFrameMetadataCreator import DataFrameMetadataCreator
from similarity.DataFrameMetadata import DataFrameMetadata
from collections import defaultdict

## Load Data from folder data

In [26]:
database, names = f.load__csv_files_from_folder("../../data")

## Create metadata for loaded data
- it contains column embeddings

In [27]:
metadata : dict[str, DataFrameMetadata] = defaultdict()
for dataframe, name in zip(database, names):
    metadata[name] = DataFrameMetadataCreator(dataframe).create_column_embeddings().get_metadata()
    

## Adding more metadata
- correlation, embeddings, column kind, categorical metadata
- create two lists one for metadata_all one for categorical 

In [28]:
from similarity.Types import DataKind
from similarity.DataFrameMetadata import DataFrameMetadata
from collections import defaultdict
from itertools import compress

categorical_all =  list()
metadata_all = list()
metadata : dict[str, DataFrameMetadata] = defaultdict()
for dataframe, name in zip(database, names):
    metadata[name] = DataFrameMetadataCreator(dataframe).compute_correlation(0.5).create_column_embeddings().compute_column_kind().get_metadata()
    categorical_all.append(dataframe[list(compress(metadata[name].column_names, metadata[name].get_column_names_by_kind(DataKind.CATEGORICAL)))])
    metadata_all.append(metadata[name].categorical_metadata)

In [36]:
print(metadata_all[0].keys())
print(categorical_all[0].keys())


dict_keys(['manufacturer', 'model', 'reg_expiration', 'reg_city', 'reg_state', 'mph'])
Index(['Unnamed: 0', 'flight', 'tail_number', 'long', 'lat', 'alt'], dtype='object')


## Add cosine similarity

In [30]:
import numpy as np


def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

## Compute similarity score

In [31]:
def compute_similarity_score(similarity_matrix: defaultdict): ## todo test some other methods
    res = 0
    count = 0
    trashold = 0.7
    length = 0
    for i in similarity_matrix.values():
        avg = sum(i)/len(i)
        length = len(i)
        if max(i) > trashold:
            count += 1
        res += max(i)
    return count, res/len(similarity_matrix) * (count/len(similarity_matrix))

In [32]:
def get_ratio(count1: int, count2: int):
    if count1 > count2:
        return count1/count2
    else:
        return count2/count1

## Compare all tables 

In [33]:

from comparing import SimilarityData, SimilarityStruct

start_time = time.perf_counter()
result = list()
for category_metadata in metadata_all:
    table_data = SimilarityData()
    for to_compare_metadata, to_compare_table_name in zip( metadata_all, names):
        if True: #category_metadata != to_compare_metadata:
            for column_name, column in category_metadata.items():
                for column_compare_name, column_to_compare in to_compare_metadata.items():
                    simil_matrix = defaultdict(list)
                    category_to_compare_embed = column_to_compare.category_embedding
                    for category, name in zip(column.category_embedding, column.categories):
                        for category_to_compare in category_to_compare_embed:
                            simil_matrix[name].append(cosine(category, category_to_compare))     
                    count, score = compute_similarity_score(simil_matrix)
                    similarity = CategoricalSimilarity(
                        categories_ratio=get_ratio(column.count_categories, column_to_compare.count_categories),
                        count_similar=count,
                        similarity_score=score
                    )
                    similarity_struct = SimilarityStruct(categorical_similarity=similarity,
                      column_name=column_compare_name,
                      table_name=to_compare_table_name                                  )
                    table_data.similarity_columns[column_name].append(similarity_struct)
    result.append(table_data)
    print("Table done")
    
finish_time = time.perf_counter()
print("Program finished in {} seconds".format(finish_time-start_time))     

Table done
Table done
Table done
Table done
Table done
Table done
Table done
Table done
Table done
Table done
Table done
Program finished in 198.6132413999876 seconds


## Prints all columns compared with all columns
columnName1 : columnName2 = similarity
**Similarity** is between 0 and 1, 1 is most similar

In [34]:
for r in result:
    for i in r.similarity_columns:
        for line in r.similarity_columns[i]:
            print(f"{i} : {line.column_name} = {line.categorical_similarity.similarity_score} ")

manufacturer : manufacturer = 1.0000000183398907 
manufacturer : model = 0.5464026626587334 
manufacturer : reg_expiration = 0.010786318452243171 
manufacturer : reg_city = 0.4797957598116644 
manufacturer : reg_state = 0.13297156964871545 
manufacturer : mph = 0.13149528194682875 
manufacturer : Company = 0.7134503035931502 
manufacturer : Engine Type = 0.16134465361458789 
manufacturer : THR = 0.11696489670846894 
manufacturer : Vstall = 0.10782416831707277 
manufacturer : make = 0.22465452683736212 
manufacturer : model = 0.35393765317915105 
manufacturer : fuel = 0.057482234696793 
manufacturer : gear = 0.05198789078645167 
manufacturer : offerType = 0.1110352066025022 
manufacturer : hp = 0.13827815738463028 
manufacturer : year = 0.0 
manufacturer : fuel_type = 0.0 
manufacturer : no_cylinder = 0.06925818667087562 
manufacturer : seating_capacity = 0.04788817878331793 
manufacturer : transmission_type = 0.04898879261662532 
manufacturer : body_type = 0.09183869564339109 
manufact